In [1]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nf
from netCDF4 import Dataset
%matplotlib inline
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import ast,gc
from copy import deepcopy

# Custom packages
import read_config
from util.data_process import read_vars, proc_dataset
from util.models import performance_scores,train_baseline,causal_settings,train_PC1

/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/miniconda3/envs/ships/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Read configuration file
config_set = read_config.read_config()
# Define Target
if int(config_set['target_lag'])==4:
    target='delv24'

seed=100

In [3]:
config_set['basin']

'NA'

In [3]:
for seed in seeds:
    # Process the filted TC list in the config file
    TC_tofilt_list = ast.literal_eval(config_set['TCfilt'])
    # Get the names of the remaining TCs
    filt_TClist = read_vars.remove_storms(trackpath=config_set['track_path'],
                                          basinID=config_set['basin'],
                                          yearmin=int(config_set['start_year']),
                                          yearmax=int(config_set['end_year']),
                                          remove_set=TC_tofilt_list
                                         )
    # Process the variable names to remove in the config file
    ERA5_dropvarname = ast.literal_eval(config_set['ERA5_dropvarname'])
    # Read saved ERA5 csvs]
    storeERA5 = read_vars.read_ERA5_csv(startyear=int(config_set['start_year']),
                                        endyear=int(config_set['end_year']),
                                        vars_path=config_set['vars_path'],
                                        filted_TCnames=filt_TClist,
                                        suffixlist=['obsw_dwmax','tigramite_6hr'],
                                        era5_dropvar=ERA5_dropvarname
                                       )
    
    storeERA5_PRIMED = read_vars.read_TCPRIMED_df(startyear=int(config_set['start_year']),
                                                  endyear=int(config_set['end_year']),
                                                  ERA5dict=storeERA5,
                                                  filted_TCnames=filt_TClist,
                                                  PRIMEDpath=config_set['PRIMED_path'],
                                                  PRIMEDlevels=ast.literal_eval(config_set['PRIMED_levels'])
                                                 )
    
    storeERA5_all = read_vars.create_ERA5_df(startyear=int(config_set['start_year']),
                                             endyear=int(config_set['end_year']),
                                             ERA5SPS_path=config_set['ERA5SPS_path'],
                                             ERA5SPS_suffix='all_storms_ships23vars_obswmax.pkl',
                                             ERA5dict=storeERA5_PRIMED,
                                             wantvarnames=ast.literal_eval(config_set['ERA5SPS_varname']),
                                             targetname=target,
                                             filted_TCnames=filt_TClist,
                                             lagnum=int(config_set['target_lag'])
                                            )

    var_names=storeERA5_all[2001]['ALLISON'].columns.values.tolist()
    
    TC_fulllist = {}
    for year in np.linspace(int(config_set['start_year']),int(config_set['end_year']),int(config_set['end_year'])-int(config_set['start_year'])+1):
        temp = storeERA5_all[year]
        for ind,name in enumerate(temp.keys()):
            TC_fulllist[str(int(year))+'_'+name] = temp[name]

    #---------------------------------------------------------------------------------------------------------
    # ML-ready dataset
    #---------------------------------------------------------------------------------------------------------
    # Split data with a 0.15 test, 0.15 valid split
    datastorer = proc_dataset.splitdata_handler(df=TC_fulllist,
                                                method='year',
                                                seed=seed,
                                                config=config_set,
                                                testyears=[2020,2021]
                                               )
    # Remove empty storms in the data
    traincleaned = {key: datastorer['train'][key] for ind,key in enumerate(datastorer['train'].keys()) if datastorer['train'][key].shape[0]>0}
    validcleaned = {key: datastorer['valid'][key] for ind,key in enumerate(datastorer['valid'].keys()) if datastorer['valid'][key].shape[0]>0}
    testcleaned = {key: datastorer['test'][key] for ind,key in enumerate(datastorer['test'].keys()) if datastorer['test'][key].shape[0]>0}
    
    # Replace original training data with the cleaned version
    datastorer_n = deepcopy(datastorer)
    
    # Replace
    datastorer_n['train'] = traincleaned
    datastorer_n['valid'] = validcleaned
    datastorer_n['test'] = testcleaned

    # Get smoothed MSLP data and argmin values
    smoothed_MSLP, MSLP_argmin = proc_dataset.proc_data(df=datastorer_n,seed=seed).smooth_and_minindices(varname='pmin',sigma=3)
    # Aligned the inputs with the minimum SLP data
    aligned_train = proc_dataset.proc_data(df=datastorer_n,seed=seed).do_data_align(datastorer_n['train'],MSLP_argmin['train'],var_names)
    aligned_valid = proc_dataset.proc_data(df=datastorer_n,seed=seed).do_data_align(datastorer_n['valid'],MSLP_argmin['valid'],var_names)
    aligned_test = proc_dataset.proc_data(df=datastorer_n,seed=seed).do_data_align(datastorer_n['test'],MSLP_argmin['test'],var_names)
    
    # Combine different TCs into a long dataset
    X,y,size = proc_dataset.df_proc_separate(aligned_train,aligned_valid,aligned_test,target)

    # Find the mean and std of the training set for normalization
    trainmean,trainstd = X['train'].dropna().mean(axis=0),X['train'].dropna().std(axis=0)

    # Data normalization
    Xnorml = proc_dataset.normalized_TCs_handler(train=aligned_train,
                                                 valid=aligned_valid,
                                                 test=aligned_test,
                                                 trainmean=trainmean,
                                                 trainstd=trainstd,
                                                 dropcol=[target],
                                                 target=target
                                                )
    var_names = Xnorml['train'][list(Xnorml['train'].keys())[0]].columns

    #---------------------------------------------------------------------------------------------------------
    # Causal
    #---------------------------------------------------------------------------------------------------------
    onlyships_lag = causal_settings.link_shipsera5(numvar=aligned_train[list(aligned_train.keys())[0]].shape[1],
                                                   lag=int(config_set['target_lag']),
                                                   target_ind=[0],
                                                   ships_ind=23
                                                  )

    results = []
    for pc_alpha in tqdm([0.75]):
        Xnorml_c = {'train': {ind: np.asarray(Xnorml['train'][key].replace(np.nan,-999.0)) for ind,key in enumerate(Xnorml['train'].keys())},
                    'valid': {ind: np.asarray(Xnorml['valid'][key].replace(np.nan,-999.0)) for ind,key in enumerate(Xnorml['valid'].keys())},
                    'test': {ind: np.asarray(Xnorml['test'][key].replace(np.nan,-999.0)) for ind,key in enumerate(Xnorml['test'].keys())}
                   }
        result = train_PC1.Pipeline(Xnorml_c['train'],
                                    pc_alpha,
                                    pc_type='run_pcstable',
                                    tau_min0=int(config_set['tau_min']),
                                    tau_max0=int(config_set['tau_max']),
                                    var_name=var_names,
                                    link_assumptions=onlyships_lag).run_tigramite()
        del Xnorml_c
        gc.collect()
        results.append(result)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 511.99it/s]


In [4]:
# Process the variable names to remove in the config file
ERA5_dropvarname = ast.literal_eval(config_set['ERA5_dropvarname'])
# Read saved ERA5 csvs
storeERA5 = read_vars.read_ERA5_csv(startyear=int(config_set['start_year']),endyear=int(config_set['end_year']),vars_path=config_set['vars_path'],
                                      filted_TCnames=filt_TClist,suffixlist=['obsw_dwmax','tigramite_6hr'],era5_dropvar=ERA5_dropvarname)

22it [00:02,  9.61it/s]


In [5]:
storeERA5_PRIMED = read_vars.read_TCPRIMED_df(startyear=int(config_set['start_year']),
                                              endyear=int(config_set['end_year']),
                                              ERA5dict=storeERA5,
                                              filted_TCnames=filt_TClist,
                                              PRIMEDpath=config_set['PRIMED_path'],
                                              PRIMEDlevels=ast.literal_eval(config_set['PRIMED_levels'])
                                             )

22it [00:13,  1.57it/s]


In [6]:
storeERA5_all = read_vars.create_ERA5_df(startyear=int(config_set['start_year']),
                                         endyear=int(config_set['end_year']),
                                         ERA5SPS_path=config_set['ERA5SPS_path'],
                                         ERA5SPS_suffix='all_storms_ships23vars_obswmax.pkl',
                                         ERA5dict=storeERA5_PRIMED,
                                         wantvarnames=ast.literal_eval(config_set['ERA5SPS_varname']),
                                         targetname=target,
                                         filted_TCnames=filt_TClist,
                                         lagnum=int(config_set['target_lag']))

22it [00:00, 57.70it/s]


In [7]:
var_names=storeERA5_all[2001]['ALLISON'].columns.values.tolist()
    
TC_fulllist = {}
for year in np.linspace(int(config_set['start_year']),int(config_set['end_year']),int(config_set['end_year'])-int(config_set['start_year'])+1):
    temp = storeERA5_all[year]
    for ind,name in enumerate(temp.keys()):
        TC_fulllist[str(int(year))+'_'+name] = temp[name]

In [8]:
#---------------------------------------------------------------------------------------------------------
# ML-ready dataset
#---------------------------------------------------------------------------------------------------------
# Split data with a 0.15 test, 0.15 valid split
datastorer = proc_dataset.splitdata_handler(df=TC_fulllist,
                                            method='year',
                                            seed=seed,
                                            config=config_set,
                                            testyears=[2020,2021]
                                           )
# Remove empty storms in the data
traincleaned = {key: datastorer['train'][key] for ind,key in enumerate(datastorer['train'].keys()) if datastorer['train'][key].shape[0]>0}
validcleaned = {key: datastorer['valid'][key] for ind,key in enumerate(datastorer['valid'].keys()) if datastorer['valid'][key].shape[0]>0}
testcleaned = {key: datastorer['test'][key] for ind,key in enumerate(datastorer['test'].keys()) if datastorer['test'][key].shape[0]>0}
    
# Replace original training data with the cleaned version
datastorer_n = deepcopy(datastorer)
    
# Replace
datastorer_n['train'] = traincleaned
datastorer_n['valid'] = validcleaned
datastorer_n['test'] = testcleaned

In [9]:
# Get smoothed MSLP data and argmin values
smoothed_MSLP, MSLP_argmin = proc_dataset.proc_data(df=datastorer_n,seed=seed).smooth_and_minindices(varname='pmin',sigma=3)
# Aligned the inputs with the minimum SLP data
aligned_train = proc_dataset.proc_data(df=datastorer_n,seed=seed).do_data_align(datastorer_n['train'],MSLP_argmin['train'],var_names)
aligned_valid = proc_dataset.proc_data(df=datastorer_n,seed=seed).do_data_align(datastorer_n['valid'],MSLP_argmin['valid'],var_names)
aligned_test = proc_dataset.proc_data(df=datastorer_n,seed=seed).do_data_align(datastorer_n['test'],MSLP_argmin['test'],var_names)
    
# Combine different TCs into a long dataset
X,y,size = proc_dataset.df_proc_separate(aligned_train,aligned_valid,aligned_test,target)

# Find the mean and std of the training set for normalization
trainmean,trainstd = X['train'].dropna().mean(axis=0),X['train'].dropna().std(axis=0)

# Data normalization
Xnorml = proc_dataset.normalized_TCs_handler(train=aligned_train,
                                             valid=aligned_valid,
                                             test=aligned_test,
                                             trainmean=trainmean,
                                             trainstd=trainstd,
                                             dropcol=[target],
                                             target=target
                                            )
var_names = Xnorml['train'][list(Xnorml['train'].keys())[0]].columns

In [12]:
#---------------------------------------------------------------------------------------------------------
# Causal
#---------------------------------------------------------------------------------------------------------
onlyships_lag = causal_settings.link_shipsera5(numvar=aligned_train[list(aligned_train.keys())[0]].shape[1],
                                               lag=int(config_set['target_lag']),
                                               target_ind=[0],
                                               ships_ind=23
                                              )

In [51]:
results = []
for pc_alpha in tqdm([0.75]):
    Xnorml_c = {'train': {ind: np.asarray(Xnorml['train'][key].replace(np.nan,-999.0)) for ind,key in enumerate(Xnorml['train'].keys())},
                'valid': {ind: np.asarray(Xnorml['valid'][key].replace(np.nan,-999.0)) for ind,key in enumerate(Xnorml['valid'].keys())},
                'test': {ind: np.asarray(Xnorml['test'][key].replace(np.nan,-999.0)) for ind,key in enumerate(Xnorml['test'].keys())}
               }
    result = train_PC1.Pipeline(Xnorml_c['train'],
                                pc_alpha,
                                pc_type='run_pcstable',
                                tau_min0=int(config_set['tau_min']),
                                tau_max0=int(config_set['tau_max']),
                                var_name=var_names,
                                link_assumptions=onlyships_lag).run_tigramite()
    del Xnorml_c
    gc.collect()
    results.append(result)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:59<00:00, 59.09s/it]


In [52]:
result[0]

[(1, -4),
 (191, -4),
 (0, -4),
 (244, -4),
 (19, -4),
 (11, -4),
 (169, -4),
 (16, -4),
 (10, -4),
 (149, -4),
 (50, -4),
 (241, -4),
 (92, -4),
 (39, -4),
 (175, -4),
 (128, -4),
 (12, -4),
 (138, -4),
 (31, -4),
 (67, -4),
 (28, -4),
 (21, -4),
 (9, -4),
 (184, -4),
 (204, -4),
 (155, -4),
 (131, -4),
 (164, -4),
 (152, -4),
 (23, -4),
 (133, -4),
 (73, -4),
 (113, -4),
 (106, -4),
 (165, -4),
 (65, -4),
 (222, -4)]

# No causal selection

In [18]:
ytrain = np.concatenate([np.asarray(Xnorml['train'][key].dropna()[target]) for key in Xnorml['train'].keys()],axis=0)
Xtrain = np.concatenate([np.asarray(Xnorml['train'][key].dropna().drop(columns=[target])) for key in Xnorml['train'].keys()],axis=0)
yvalid = np.concatenate([np.asarray(Xnorml['valid'][key].dropna()[target]) for key in Xnorml['valid'].keys()],axis=0)
Xvalid = np.concatenate([np.asarray(Xnorml['valid'][key].dropna().drop(columns=[target])) for key in Xnorml['valid'].keys()],axis=0)
ytest = np.concatenate([np.asarray(Xnorml['test'][key].dropna()[target]) for key in Xnorml['test'].keys()],axis=0)
Xtest = np.concatenate([np.asarray(Xnorml['test'][key].dropna().drop(columns=[target])) for key in Xnorml['test'].keys()],axis=0)

In [19]:
Xnorml_nocausal = {'train': Xtrain, 'valid': Xvalid, 'test': Xtest}
y = {'train': ytrain, 'valid': yvalid, 'test': ytest}
regr = train_baseline.train_baseline_MLR(Xnorml_nocausal,y)

MLR_scoreboard = performance_scores.scoreboard(regr).store_scores(Xnorml_nocausal,y)
MLR_scoreboard['train']['r2'],MLR_scoreboard['valid']['r2'],MLR_scoreboard['test']['r2']

(0.3190335148694612, 0.25704625858822294, 0.3399466154523325)

In [21]:
Xvalid.shape

(1758, 254)

# With causal

In [32]:
def benchmark_causal(PC1_results=None,Xnorml=None):
    causal_predictor_list = [var_names[i] for i in [obj[0] for obj in PC1_results[0]]]
    while target in causal_predictor_list: 
        causal_predictor_list.remove(target)
        
    Xtrain_causal = np.concatenate([np.asarray(Xnorml['train'][key].dropna()[causal_predictor_list]) for key in Xnorml['train'].keys()],axis=0)
    Xvalid_causal = np.concatenate([np.asarray(Xnorml['valid'][key].dropna()[causal_predictor_list]) for key in Xnorml['valid'].keys()],axis=0)
    Xtest_causal = np.concatenate([np.asarray(Xnorml['test'][key].dropna()[causal_predictor_list]) for key in Xnorml['test'].keys()],axis=0)
    
    Xnorml_causal = {'train': Xtrain_causal, 'valid': Xvalid_causal, 'test': Xtest_causal}
    regr_causal = train_baseline.train_baseline_MLR(Xnorml_causal,y)
    return performance_scores.scoreboard(regr_causal).store_scores(Xnorml_causal,y),Xnorml_causal,regr_causal

In [53]:
scores,Xs,regr = [],[],[]
for obj in results:
    score,X,regrz = benchmark_causal(PC1_results=obj,Xnorml=Xnorml)
    scores.append(score)
    Xs.append(X)
    regr.append(regrz)

In [50]:
[obj['train']['r2'] for obj in scores],[obj['valid']['r2'] for obj in scores],[obj['test']['r2'] for obj in scores]

([0.22758870781273133], [0.2620004791744047], [0.24884417564138772])

In [54]:
[obj['train']['r2'] for obj in scores],[obj['valid']['r2'] for obj in scores],[obj['test']['r2'] for obj in scores]

([0.2345899268984507], [0.2580179531419897], [0.25898521559471577])